In [3]:
import fitz 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
%pip install sentence_transformers

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached sentence_transformers-4.0.2-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached sentence_transformers-3.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached sentence_transformers-3.3.0-py3-none-any.whl.metadata (10 kB)
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a 

In [5]:
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(extracted_text)



model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)



dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

query = "What is the main point of the document?"
query_embedding = model.encode([query])

D, I = index.search(np.array(query_embedding), k=5)
retrieved_chunks = [chunks[i] for i in I[0]]

from openai import OpenAI

context = "\n".join(retrieved_chunks)
prompt = f"Answer the question based on the context:\n\nContext:\n{context}\n\nQuestion: {query}"

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}]
)

answer = response['choices'][0]['message']['content']



NameError: name 'RecursiveCharacterTextSplitter' is not defined